<a href="https://colab.research.google.com/github/PrishaAggarwal/Machine_Learning_Assignments/blob/main/ML_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#now lets start assignment_3

#Q1
#we have some dataset and we have to implement k fold cross validation on it

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold,train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

#load the dataset
df = pd.read_csv("/content/USA_Housing.csv")
print(df.head())

#convert the dataset into input features and output variables
X=df.drop('Price',axis=1).values
y=df['Price'].values.reshape(-1,1) #target as column vector

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

#doing k-fold split
Kf = KFold(n_splits=5,shuffle=True,random_state=42)

best_r2 = -np.inf #np.inf means negative infinity
#later when we find r2 for each fold then any real value would be higher than this
best_beta = None #beta is the coefficient
fold = 1 #counter for current fold in cross-validation

for train_index,test_index in Kf.split(X_scaled):
  #Kf.split(X_scaled) comes from KFold...it returns 2 arrays of indices
  #train_index : row indices for the training set in this fold
  #test_index : test indices for testing set in this fold
  X_train,X_test = X_scaled[train_index],X_scaled[test_index]
  y_train,y_test = y[train_index],y[test_index]

  #we have to add bias term to X_train and X_test ..bias term is intercept
  X_train_bias = np.hstack([np.ones((X_train.shape[0],1)),X_train])
  X_test_bias = np.hstack([np.ones((X_test.shape[0],1)),X_test])
  #we have a column of 1's in X so we have to add that since we have to compute beta_0
  #no.ones upar wala will create a vector of number of rows in trainingx1 of 1's
  #hstack is horizontal stack ...it adds column of 1's to left side of X_train

  #compute beta using formula for b = (X^T X)^-1 X^T y
  beta = np.linalg.inv(X_train_bias.T @ X_train_bias) @ X_train_bias.T @ y_train
  #the beta we get from here is for this particular fold
  #we get a beta matrix here
  print(f"Fold{fold} Beta : {beta}")

  # lets predict the test set
  y_pred = X_test_bias @ beta

  #calculate R2 score
  r2 = r2_score(y_test,y_pred)
  print(f"Fold{fold} R2 Score : {r2:.4f}")

  #update the best beta if current beta is higher
  if r2 > best_r2:
    best_r2 = r2
    best_beta = beta

  fold += 1

print("best r2",best_r2)
print("best_bets",best_beta)

#part e
#we have to make predictions based on best beta matrix

#train-test split
X_train1,X_test1,y_train1,y_test1 = train_test_split(X_scaled,y,test_size=0.3,random_state=42)

#adding bias
X_train_bias1 = np.hstack([np.ones((X_train1.shape[0],1)),X_train1])
X_test_bias1 = np.hstack([np.ones((X_test1.shape[0],1)),X_test1])

#use best_beta
y_pred_train1 = X_train_bias1 @ best_beta
y_pred_test1 = X_test_bias1 @ best_beta

#evaluate performance
r2_train1 = r2_score(y_train1,y_pred_train1)
r2_test1 = r2_score(y_test1,y_pred_test1)

print("R2 score on training set",r2_train1)
print("r2 on test set",r2_test1)
print("best beta",best_beta.flatten())

   Avg. Area Income  Avg. Area House Age  Avg. Area Number of Rooms  \
0       79545.45857             5.682861                   7.009188   
1       79248.64245             6.002900                   6.730821   
2       61287.06718             5.865890                   8.512727   
3       63345.24005             7.188236                   5.586729   
4       59982.19723             5.040555                   7.839388   

   Avg. Area Number of Bedrooms  Area Population         Price  
0                          4.09      23086.80050  1.059034e+06  
1                          3.09      40173.07217  1.505891e+06  
2                          5.13      36882.15940  1.058988e+06  
3                          3.26      34310.24283  1.260617e+06  
4                          4.23      26354.10947  6.309435e+05  
Fold1 Beta : [[-1419665.22833691]
 [ 1946644.09703889]
 [ 1132046.46723314]
 [  899978.61379173]
 [   10981.69925197]
 [ 1060509.56556484]]
Fold1 R2 Score : 0.9180
Fold2 Beta : [[-142

In [ ]:
#Q2

#training set : used to fit the model
#test set : used to evaluate the final performance of the model
#validation set : a seperate set used to tune the model or choose the best hyperparameters

#so taking in the same dataset from Q1, lets divide the dataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train2,X_temp2,y_train2,y_temp2 = train_test_split(X,y,test_size = 0.44 ,random_state=42,shuffle=True)
#we took temp since we will further divide temp into validation and testing data
#shuffle ensures that dataset is shuffled beflore splitting and it should not be biased by taking rows in order

# 0.001,0.01,0.1,1 are 3 diff alpha learning rates
#so i had a ques how we were in the above ques without gradient descent
#basically in above equation we were just finding beta values through linear equation
#we dont do iterations since this works for small to medium datasets

#gradient descent is used for large datasets
#we iterate again and again to find best beta posible

#lets do the 3rd split
X_val2,X_test2,y_val2,y_test2 = train_test_split(X_temp2,y_temp2,test_size=(0.3/0.44),random_state=42,shuffle=True)

#pre-processing of data is always done after spliting the data into training,testing and validation
#checking the shape of each set we divided once to ensure its correct
m=X.shape[0]
print(f"Total rows : {m}")
print("Training rows",X_train2.shape[0])
print("Validation rows",X_val2.shape[0])
print("Test rows",X_test.shape[0])

scaler = StandardScaler()
X_train_scaled2 = scaler.fit_transform(X_train2)
X_val_scaled2 = scaler.transform(X_val2)
X_test_scaled2 = scaler.transform(X_test2)

#adding bias wala column now
def add_bias(X_arr):
  return np.hstack([np.ones((X_arr.shape[0],1)),X_arr])

X_train_bias2 = add_bias(X_train_scaled2)
X_val_bias2 = add_bias(X_val_scaled2)
X_test_bias2 = add_bias(X_test_scaled2)

#now lets move to gradient descent
def gradient_descent(Xb,y,lr,iters=1000):
  '''
  Xb : (m,n+1) with bias column
  returns beta(n+1,1)
  '''
  m=Xb.shape[0]
  final_n = Xb.shape[1]
  beta=np.zeros((final_n,1))

  for i in range(iters):
    preds = Xb @ beta
    error = preds - y
    #gradient : (n+1,1) = (1/m) * Xb^T @ error
    gradient = (Xb.T @ error) / m
    beta = beta - lr*gradient
  return beta

learning_rates = [0.001,0.01,0.1,1]
iters=1000

results = {} #we have to store results per learning rate

for lr in learning_rates:
  beta_lr = gradient_descent(X_train_bias2,y_train2,lr,iters)

  #predictions
  y_training_pred = X_train_bias2 @ beta_lr
  y_val_pred = X_val_bias2 @ beta_lr
  y_test_pred = X_test_bias2 @ beta_lr

  #computing R2 for each split
  r2_train2 = r2_score(y_train2,y_training_pred)
  r2_val2 = r2_score(y_val2,y_val_pred)
  r2_test2 = r2_score(y_test2,y_test_pred)

  results[lr] = {
      "beta" : beta_lr,
      "r2_train" : r2_train2,
      "r2_val" : r2_val2,
      "r2_test" : r2_test2
  }
  print(f"LR = {lr:>6} | R2_train={r2_train2:.4f} | r2_val = {r2_val2:.4f} | r2_test = {r2_test2:.4f} ")
  # :>6 means
  # > means right align the value
  # 6 means use total width of 6  characters

best_lr = max(results.keys(),key=lambda a : results[a]["r2_val"])
#results.keys gets the list of all learning rates
#here we are comparing the results
#take each learning rate a
#see inside dictionary and take out the r2_score
#then we get the max r2_score
best_info = results[best_lr]

print("Best learning rate acc to validation R2", best_lr)
print("Best validation r2",best_info["r2_val"])
print("Corresponding train R2:", best_info["r2_train"])
print("Corresponding test R2:", best_info["r2_test"])
print("Best beta (coefficients):\n", best_info["beta"].flatten())

Total rows : 5000
Training rows 2800
Validation rows 700
Test rows 1000
LR =  0.001 | R2_train=-0.7872 | r2_val = -1.0005 | r2_test = -0.9212 
LR =   0.01 | R2_train=0.9197 | r2_val = 0.9199 | r2_test = 0.9134 
LR =    0.1 | R2_train=0.9197 | r2_val = 0.9200 | r2_test = 0.9134 
LR =      1 | R2_train=0.9197 | r2_val = 0.9200 | r2_test = 0.9134 
Best learning rate acc to validation R2 0.1
Best validation r2 0.9199649194854793
Corresponding train R2: 0.9196588895678885
Corresponding test R2: 0.9134494051887397
Best beta (coefficients):
 [1225106.34781021  231827.54854547  166006.22902472  120763.07797071
    2922.26769971  152609.02782229]


In [ ]:
#Q3

import pandas as pd

#Part 1
columns = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]

df1 =pd.read_csv("/content/imports-85.csv",names=columns)
df1.replace("?",np.nan,inplace=True)

print(df1.head())

#Part 2
df1 = df1.dropna(subset=["price"]) #dropna removes that have missing values
#by default dropna looks at all columns so here we tell it to look at only price

#reset the indec after dropping
df1.reset_index(drop=True,inplace=True)
#reset_index resets the index
#drop=True means doesnt keep old index as seperate column

print(df1.head())

#checking number of null values in each column
print(df1.isnull().sum())
print(df1.info())

'''
#so columns with isnull are normalized_losses,num_doors,bore,stroke,horsepower, peak_rpm
columns_nan = ["bore","stroke","horsepower","peak_rpm"]
cate_nan = ['num_doors',"normalized_losses"]

#we will replace the nan values with central tendency i.e median
#but num_door is categorial so there we will use mode
df1[columns_nan] = df1[columns_nan].apply(lambda col : col.fillna(col.median()))

#for num_door
# df1['cate_nan'] = df1['cate_nan'].fillna(df1['cate_nan'].mode()[0])
#above code is wrong since this line is literally lookinng for column named 'cate_nan

for col in cate_nan:
    df1[col] = df1[col].fillna(df1[col].mode()[0])

#all the above code can be used for small dataset
#here u have to check the type of column if it is object or numeric
#in this dataset it couldnt work since mostly numeric values are objects
'''

#so lets convert the numeric values into object to get proper replacement of nan
#and then apply the easier code given below instead of making a list for numeric and categorical columns

numerical_cols1 = ["normalized_losses","wheel_base","length","width","height",
                "curb_weight","engine_size","bore","stroke","compression_ratio",
                "horsepower","peak_rpm","city_mpg","highway_mpg","price"]

df1[numerical_cols1] = df1[numerical_cols1].apply(pd.to_numeric, errors="coerce")
#errors="coerce" ensures that non-numeric values (like NaN from "?") are safely converted to NaN.

df1=df1.fillna(df1.median(numeric_only=True))
for col in df1.select_dtypes(include="object").columns:
  df1[col] = df1[col].fillna(df1[col].mode()[0]) #.mode() returns all of multiple modes in series if there are multiple so we take 1st item only if multiple mode are there

print(df1.isnull().sum())

   symboling normalized_losses         make fuel_type aspiration num_doors  \
0          3               NaN  alfa-romero       gas        std       two   
1          3               NaN  alfa-romero       gas        std       two   
2          1               NaN  alfa-romero       gas        std       two   
3          2               164         audi       gas        std      four   
4          2               164         audi       gas        std      four   

    body_style drive_wheels engine_location  wheel_base  ...  engine_size  \
0  convertible          rwd           front        88.6  ...          130   
1  convertible          rwd           front        88.6  ...          130   
2    hatchback          rwd           front        94.5  ...          152   
3        sedan          fwd           front        99.8  ...          109   
4        sedan          4wd           front        99.4  ...          136   

   fuel_system  bore  stroke compression_ratio horsepower  peak_rpm 

In [ ]:
#Q3
#Part 3
#now we are gonna perform some steps of pre-processing

#part (i)
#we have to convert words into numbers in 2 columns
#first lets find out the unique values in both columns

print(df1["num_doors"].unique())
print(df1["num_cylinders"].unique())

#lets do mapping for both of them
num_doors_map = {
    "four":4,
    "two":2,
}

num_cylinders_map = {
    "four":4,
    "six":6,
    "five":5,
    "eight":8,
    "two":2,
    "twelve":12,
    "three":3
}

df1["num_cylinders"] = df1["num_cylinders"].map(num_cylinders_map)
df1["num_doors"] = df1["num_doors"].map(num_doors_map)

print(df1.head())

#part(ii)
#dummy encoding scheme is one-hot encoding

print(df1["body_style"].unique())
print(df1["drive_wheels"].unique())

df1= pd.get_dummies(df1,columns=["body_style"],drop_first=True,dtype=int)
df1= pd.get_dummies(df1,columns=["drive_wheels"],drop_first=True,dtype=int)

print(df1.head())

#part(iii)
print(df1["make"].unique())
print(df1["aspiration"].unique())
print(df1["engine_location"].unique())
print(df1["fuel_type"].unique())

#so now we have to do label encoding
#it can be done in 2 ways
#one of them when you convert catgories into numbers ....like 1-20 if category
#another is manual label encoding in which u map with categorical values acc to label u want
#we are doing 1st one using sklearn since we mostly try to convert the whole data in category
from sklearn.preprocessing import LabelEncoder

label_cols = ["make","aspiration","engine_location","fuel_type"]

le=LabelEncoder()
for col in label_cols:
  df1[col] = le.fit_transform(df1[col])

print(df1.head())

['two' 'four']
['four' 'six' 'five' 'three' 'twelve' 'two' 'eight']
   symboling  normalized_losses         make fuel_type aspiration  num_doors  \
0          3              115.0  alfa-romero       gas        std          2   
1          3              115.0  alfa-romero       gas        std          2   
2          1              115.0  alfa-romero       gas        std          2   
3          2              164.0         audi       gas        std          4   
4          2              164.0         audi       gas        std          4   

    body_style drive_wheels engine_location  wheel_base  ...  engine_size  \
0  convertible          rwd           front        88.6  ...          130   
1  convertible          rwd           front        88.6  ...          130   
2    hatchback          rwd           front        94.5  ...          152   
3        sedan          fwd           front        99.8  ...          109   
4        sedan          4wd           front        99.4  ...      

In [ ]:
#Q3
#Part 3

#part(iv)
#we are gonna use the where condition here

import numpy as np
df1["fuel_system"] = np.where(df1["fuel_system"] == "pfi",1,0)

#part(v)
df1["engine_type"] = np.where(df1["engine_type"] == "ohc",1,0)

print(df1.head())

   symboling  normalized_losses  make  fuel_type  aspiration  num_doors  \
0          3              115.0     0          1           0          2   
1          3              115.0     0          1           0          2   
2          1              115.0     0          1           0          2   
3          2              164.0     1          1           0          4   
4          2              164.0     1          1           0          4   

   engine_location  wheel_base  length  width  ...  peak_rpm  city_mpg  \
0                0        88.6   168.8   64.1  ...    5000.0        21   
1                0        88.6   168.8   64.1  ...    5000.0        21   
2                0        94.5   171.2   65.5  ...    5000.0        19   
3                0        99.8   176.6   66.2  ...    5500.0        24   
4                0        99.4   176.6   66.4  ...    5500.0        18   

   highway_mpg  price  body_style_hardtop  body_style_hatchback  \
0           27  13495                

In [ ]:
#Q3
#Part 4

y3 = df1["price"]
X3 = df1.drop("price",axis=1)

#we need info since now we have to scale the columns
df1.info()
#we need to know which columns need to be normalized
# Columns to normalize (numeric but not binary)
binary_cols1 = [col for col in numerical_cols1 if df1[col].nunique() == 2]
cols_to_normalize = [col for col in numerical_cols1 if col not in binary_cols1]
cols_to_normalize.remove('price')
#we had to drop price since we dont have to normalize the target value
#done assin the above line back to cols_to_normalize since it returns none
#and hence the list becomes none

print("Columns to normalize:", cols_to_normalize)

from sklearn.preprocessing import MinMaxScaler

scaler =  MinMaxScaler()
X3[cols_to_normalize] = scaler.fit_transform(X3[cols_to_normalize])

print(X3.head())

#Part 5
from sklearn.model_selection import train_test_split

X3_train,X3_test,y3_train,y3_test = train_test_split(X3,y3,test_size=0.3,random_state=42)

from sklearn.linear_model import LinearRegression

lr1=LinearRegression()
lr1.fit(X3_train,y3_train)

y3_pred = lr1.predict(X3_test)

from sklearn.metrics import r2_score

r2 = r2_score(y3_test,y3_pred)
print("R2 score",r2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   symboling             201 non-null    int64  
 1   normalized_losses     201 non-null    float64
 2   make                  201 non-null    int64  
 3   fuel_type             201 non-null    int64  
 4   aspiration            201 non-null    int64  
 5   num_doors             201 non-null    int64  
 6   engine_location       201 non-null    int64  
 7   wheel_base            201 non-null    float64
 8   length                201 non-null    float64
 9   width                 201 non-null    float64
 10  height                201 non-null    float64
 11  curb_weight           201 non-null    int64  
 12  engine_type           201 non-null    int64  
 13  num_cylinders         201 non-null    int64  
 14  engine_size           201 non-null    int64  
 15  fuel_system           2

In [ ]:
#Q3
#Part 6

#we are applying PCA directly since are features are already normalized
from sklearn.decomposition import PCA

pca = PCA(n_components=5) #keeping top 5 components
X_pca = pca.fit_transform(X3)

print("Explained variance ratio:", pca.explained_variance_ratio_)
#it tells the what fraction of total variance in dataset is captured by that pc
print("X_pca shape",X_pca.shape)

#now lets train the model on reduced dimensions
x_train4,x_test4,y_train4,y_test4 = train_test_split(X_pca,y3,test_size=0.3,random_state=42)

lr2 = LinearRegression()
lr2.fit(x_train4,y_train4)

y_pred4 = lr2.predict(x_test4)

from sklearn.metrics import r2_score

r2_pca = r2_score(y_test4,y_pred4)
print("R2 score",r2_pca)

diff = r2-r2_pca
print("Difference in R2 scores",diff)
if(diff>0):
  print("Since diff is positive so model performance decreases since information loss happened ")
else:
  print("The model performance improved")

Explained variance ratio: [0.87433839 0.05275358 0.02921721 0.01050539 0.00981677]
X_pca shape (201, 5)
R2 score 0.7140062020488842
Difference in R2 scores 0.1629980223807228
Since diff is positive so model performance decreases since information loss happened 
